In [1]:
!sudo pip install --upgrade transformers

In [1]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import json
#from Gpt4_OpenAI_Prompt import system_msg,context,postprocess
#from prompt1 import system_msg
import transformers
print(transformers.__version__)

4.34.1


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla T4


In [16]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float32"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "fp4"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Load the entire model on the GPU 0
device_map = {"": 0}


In [17]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
model_mistral = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,
    device_map=device_map)
tokenizer_mistral = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
def gereneate_output_mistral(prompt):
    #prompt = json.dumps(prompt)
    model_input = tokenizer_mistral(prompt, return_tensors="pt").to("cuda")
    #model.eval()
    response = tokenizer_mistral.decode(model_mistral.generate(**model_input, max_length=4069*2)[0], skip_special_tokens=True)
    return response

In [43]:
################################################################################
# Setting the same prompt which is used for GPT4 open AI
################################################################################

def get_user_question_for_open_ai_prompt(question):
    #return """Now Based on this following "CONTEXT" data lookup:\n""" + str(context) + "\n return a response for the question:\n" + question + "\n"
    return "\n Now detect from this question:\n" + question + "\n"

def compose_prompt_open_ai_prompt(question,base_prompt):
    return f"""[INST]<<SYS>>
            {base_prompt}
            <</SYS>>
            {get_user_question_for_open_ai_prompt(question)}
            [/INST]
            [_separator_]"""

def post_process_open_ai_prompt_response(output):
    mql = output.split("[END]")[1].split("\n\nReasoning:\n\n")[0]
    reason = output.split("[END]")[1].split("\n\nReasoning:\n\n")[1]
    print("*"*50+"MQL"+"*"*50)
    print(mql)
    print("\n"+"*"*50+"Reason"+"*"*50)
    print(reason)
    #return mql,reason

In [44]:
METRIC_CONSTRAINT= """

**Task**: Extract "COMPARISON OPERATOR," "COMPARISON VALUE," and the Targeted ENTITY

**Instructions**:

1. Read the input question carefully.
2. Determine if there is a comparison intent present, such as "greater than," "less than," or "equals to." etc, If there is a comparison intent, please proceed to extract the relevant information.
3. Extract the "COMPARISON OPERATOR" based on the detected intent. Examples of "COMPARISON OPERATORS" include "greater than," "less than," and "equals to."
4. Identify and extract the "COMPARISON VALUE" from the sentence. This should be a numerical or scalar value.
5. Detect the ENTITY (e.g., "Sales") on which these comparisons are applied. Please extract the relevant entity.
6. Provide the following information:
   - COMPARISON OPERATOR: (If detected else None) 
   - COMPARISON VALUE: (If detected else None)
   - ENTITY: (If detected else None)

**Input Sentence**: "When was the first time sales of segments was 0"

**Output**:
- COMPARISON OPERATOR: (If detected else None)
- COMPARISON VALUE: (If detected else None)
- ENTITY: Sales (If detected else None)

*Note*: If the input question does not contain any comparison intent or the entity, you can indicate that they were not detected.

"""

In [51]:
ADJECTIVE_and_TONE = """
**Task**: Extract Adjectives, Determine Intent (TONE), Identify Entity, Return RANK ADJECTIVE, and Scalar (RANK)

**Instructions**:

1. Carefully read the user question to identify any adjectives present. Adjectives can include terms like "least," "highest performing," etc.
2. Determine the intent (TONE) of the adjective. The intent can be either positive or negative.
   - If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
   - If the adjective intent is positive, always  set RANK ADJECTIVE to "TOP" and TONE to "positive."
3. If the adjective implies a quantity (e.g., "how many," "top 3"), extract that scalar value as RANK.
4. Identify the entity (e.g., "Segment") on which the detected adjective should be applied.

**Input Question**: "How many top 3 segments are performing well?"

**Output**:
- ADJECTIVE: top
- TONE: positive
- ENTITY: Segment
- RANK ADJECTIVE: TOP
- TONE: positive
- RANK: 3

**Note**: If the input question doesn't contain an adjective, doesn't imply a quantity, or if it's not clear which entity the adjective should be applied to, you can indicate that they were not detected.
"""

In [49]:
%%time
question = "show me quantities > 100K"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,METRIC_CONSTRAINT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




**Task**: Extract "COMPARISON OPERATOR," "COMPARISON VALUE," and the Targeted ENTITY

**Instructions**:

1. Read the input question carefully.
2. Determine if there is a comparison intent present, such as "greater than," "less than," or "equals to." etc, If there is a comparison intent, please proceed to extract the relevant information.
3. Extract the "COMPARISON OPERATOR" based on the detected intent. Examples of "COMPARISON OPERATORS" include "greater than," "less than," and "equals to."
4. Identify and extract the "COMPARISON VALUE" from the sentence. This should be a numerical or scalar value.
5. Detect the ENTITY (e.g., "Sales") on which these comparisons are applied. Please extract the relevant entity.
6. Provide the following information:
   - COMPARISON OPERATOR: (If detected else None) 
   - COMPARISON VALUE: (If detected else None)
   - ENTITY: (If detected else None)

**Input Sentence**: "show me quantities > 100K"

**Output**:
- COMPARISON OPERATOR: greater than
- COMPAR

In [50]:
%%time
question = "When is first time discount of phone is lesser than 0"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,METRIC_CONSTRAINT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




**Task**: Extract "COMPARISON OPERATOR," "COMPARISON VALUE," and the Targeted ENTITY

**Instructions**:

1. Read the input question carefully.
2. Determine if there is a comparison intent present, such as "greater than," "less than," or "equals to." etc, If there is a comparison intent, please proceed to extract the relevant information.
3. Extract the "COMPARISON OPERATORS" based on the detected intent. Examples of "COMPARISON OPERATORS" include "greater than," "less than," and "equals to."
4. Identify and extract the "COMPARISON VALUE" from the sentence. This should be a numerical or scalar value.
5. Detect the ENTITY (e.g., "Sales") on which these comparisons are applied. Please extract the relevant entity.
6. Provide the following information:
   - COMPARISON OPERATOR: (If detected else None) 
   - COMPARISON VALUE: (If detected else None)
   - ENTITY: (If detected else None)

**Input Sentence**: "When is first time discount of phone is lesser than 0"

**Output**:
- COMPARISON OP

In [ ]:
%%time
question = "top 2 segments and bottom 3 sub-category basis quantity"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
